In [ ]:
import json
import os

In [ ]:
output_folder = "./output_chapters/"
chapters_file_name = "chapters.json"

In [ ]:
chapters_file_path = output_folder + chapters_file_name

# Confirm that file exists
if not os.path.exists(chapters_file_path):
    print(f"Error: The file {chapters_file_path} does not exist.")
    exit(1)
else:
    print(f"File {chapters_file_path} exists.")


In [ ]:
chapters = {}
# read the json and save it as dictionary
with open(chapters_file_path, 'r') as file:
    try:
        chapters = json.load(file)
        print("JSON file loaded successfully.")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        exit(1)

In [ ]:
print(f"Number of chapters: {len(chapters)}")
# Get the chapter called "Introduction"
introduction_chapter = next(
    (chapter for chapter in chapters.values() if chapter.get("title") == "Introduction"),
    None
)

print(f"Introduction chapter:\n{introduction_chapter.get('content', '')[:500]}")

### That is ugly
We have the chapters, but we still have it divided by lines. We now need our chunks!